In [1]:
import pandas as pd
import numpy as np
from collections import Counter

# 위염 환자 중 위염으로 끝난 환자
# 위암 환자 전체

## 파일 불러오기

In [86]:
cg = pd.read_csv('chronic_gastritis_person0722(4225273).csv')

cg_df = pd.DataFrame(cg)
print(len(cg_df)) # 위염 환자 진료 데이터 수

cg_df=cg_df[['person_id','condition_start_date','gender_concept_id','year_of_birth']]
cg_df.sort_values(by=['person_id','condition_start_date'],axis=0,inplace=True)

cg_df=cg_df.drop_duplicates(['person_id'],keep='first')
print(len(cg_df)) # 위염 환자 진료 데이터 수

55565
12582


In [87]:
cg_df.columns = ['person_id','date','gender','year_of_birth']

In [88]:
cg_df

,person_id,date,gender,year_of_birth
0,69,2000-01-03,8532,1945
3,115,2000-01-18,8507,1949
4,150,2006-04-17,8532,1928
5,157,2001-01-15,8507,1946
13,270,2000-01-12,8507,1962
...,...,...,...,...
55054,1760388,2014-05-17,8532,1971
55062,1760747,2007-03-02,8532,1961
55063,1760757,1999-11-29,8532,1977
55066,1760837,1999-12-15,8532,1972


In [89]:
sm = pd.read_csv('stomachcancer_0722(443387,4095320,40948556,4095319).csv')

sm_df = pd.DataFrame(sm)

sm_df=sm_df.loc[:,['person_id','condition_start_date']]
sm_df.sort_values(by=['person_id','condition_start_date'],axis=0,inplace=True)
sm_df=sm_df.drop_duplicates(['person_id'],keep='first')
print(len(sm_df))
# 위암, 위염 진료 기록의 경우 최초 기록은 date에 저장

553


In [6]:
sm_df.columns = ['person_id','date']

In [7]:
sm_df

,person_id,date
0,8022,2008-05-16
1,19905,2014-07-05
2,25081,2011-03-21
3,31610,2015-08-09
4,36415,2017-02-23
...,...,...
648,1734314,2011-01-17
650,1737002,2009-03-04
652,1743084,2014-03-20
653,1756420,2008-04-02


In [8]:
# 위염에서 위암으로 발전한 환자 찾기

cg_persons=list(set(cg_df['person_id'].tolist()))
sm_persons=list(set(sm_df['person_id'].tolist()))

a=0
to_cancer=[]
for i in sm_persons:
    if i in cg_persons:
        a+=1
        to_cancer.append(i)
print('위암발전 환자 : %d' %a)

위암발전 환자 : 27


In [9]:
to_sm = pd.DataFrame({'person_id':to_cancer})

In [10]:
to_sm

,person_id
0,297005
1,309303
2,739433
3,1663082
4,684303
5,270704
6,1155470
7,1221108
8,25081
9,1577509


In [11]:
#to_sm.to_csv('to_cancer_kd.csv',mode='w',index=False)

In [12]:
sm_df = pd.merge(to_sm,sm_df,how='left',on=['person_id'])
sm_df # 위암 환자 중 위염에서 발전된 환자 수만 거르기

,person_id,date
0,297005,2007-02-03
1,309303,2012-05-03
2,739433,2015-04-03
3,1663082,2016-08-10
4,684303,2014-06-08
5,270704,2006-08-24
6,1155470,2016-08-05
7,1221108,2012-09-18
8,25081,2011-03-21
9,1577509,2018-12-31


In [13]:
real_cg = pd.merge(cg_df,sm_df,how='left',on=['person_id'])

# data_x : 위염 최초 진료 날짜, data_y : 위암 최초 진단 날짜

In [14]:
real_cg['date_y'] = np.where(pd.notnull(real_cg['date_y'])==True,real_cg['date_y'],real_cg['date_x'])

In [15]:
del real_cg['date_x']

In [16]:
real_cg.columns = ['person_id','gender','year_of_birth','date']
# 위암 환자의 경우, 위암 최초 진단 날짜로 변경
# 위염 환자는 위염 최초 진단 날짜

In [17]:
real_cg

,person_id,gender,year_of_birth,date
0,69,8532,1945,2000-01-03
1,115,8507,1949,2000-01-18
2,150,8532,1928,2006-04-17
3,157,8507,1946,2001-01-15
4,270,8507,1962,2000-01-12
...,...,...,...,...
12577,1760388,8532,1971,2014-05-17
12578,1760747,8532,1961,2007-03-02
12579,1760757,8532,1977,1999-11-29
12580,1760837,8532,1972,1999-12-15


In [18]:
real_cg.notnull().sum()

person_id        12582
gender           12582
year_of_birth    12582
date             12582
dtype: int64

## Gender

In [19]:
# gender - 8507 : male(0), 8532 : female(1)
real_cg = real_cg.astype({'gender':str})

concept_id = {'8507':0,'8532':1}

real_cg['gender']=real_cg['gender'].replace(concept_id)

In [20]:
real_cg

,person_id,gender,year_of_birth,date
0,69,1,1945,2000-01-03
1,115,0,1949,2000-01-18
2,150,1,1928,2006-04-17
3,157,0,1946,2001-01-15
4,270,0,1962,2000-01-12
...,...,...,...,...
12577,1760388,1,1971,2014-05-17
12578,1760747,1,1961,2007-03-02
12579,1760757,1,1977,1999-11-29
12580,1760837,1,1972,1999-12-15


## 음주습관

In [21]:
al = pd.read_csv('frequency of drinking(4266614).csv')

al_df = pd.DataFrame(al)

al_df.sort_values(by=['person_id','observation_date'],axis=0,inplace=True)

In [22]:
# 월2~3회 이하로 마신다(1), 일주일에 1~2회 마신다(2), 
# 일주일에 3~4회 마신다(3), 거의 매일 마신다(4)

al_df.loc[(al_df['value_as_number']<=3),['observation_source_value']]=1
al_df.loc[((al_df['value_as_number']<=11)&(al_df['value_as_number']>3)),['observation_source_value']]=2
al_df.loc[((al_df['value_as_number']<=19)&(al_df['value_as_number']>11)),['observation_source_value']]=3
al_df.loc[((al_df['value_as_number']<=31)&(al_df['value_as_number']>19)),['observation_source_value']]=4

In [23]:
Counter(al_df['observation_source_value'])

Counter({1: 645, 2: 760, 4: 402, 3: 165})

In [24]:
al_df

,person_id,condition_concept_id,observation_id,person_id-2,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value
1,1562,4225273,17804790,1562,4266614,2013-09-23,2013-09-23 00:00:00,38000280,3,NaN,NaN,NaN,44777658,1995.0,9557957.0,11174750.0,1,0,times/month,ach_times_month
0,1562,4225273,17731067,1562,4266614,2015-08-31,2015-08-31 00:00:00,38000280,3,NaN,NaN,NaN,44777658,6739.0,NaN,NaN,1,0,times/month,ach_times_month
6,2077,4225273,17646324,2077,4266614,2011-04-01,2011-04-01 00:00:00,38000280,4,NaN,NaN,NaN,44777658,974.0,5045742.0,5902237.0,2,0,times/month,ach_times_month
3,2077,4225273,17646393,2077,4266614,2011-04-28,2011-04-28 00:00:00,38000280,4,NaN,NaN,NaN,44777658,264.0,5045746.0,5902245.0,2,0,times/month,ach_times_month
7,2077,4225273,17646395,2077,4266614,2011-05-13,2011-05-13 00:00:00,38000280,4,NaN,NaN,NaN,44777658,264.0,5045750.0,5902249.0,2,0,times/month,ach_times_month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1966,1759469,4225273,17821324,1759469,4266614,2017-10-01,2017-10-01 00:00:00,38000280,8,NaN,NaN,NaN,44777559,7070.0,NaN,NaN,2,0,times/month,ach_times_month
1968,1759639,4225273,17767157,1759639,4266614,2015-08-20,2015-08-20 00:00:00,38000280,8,NaN,NaN,NaN,44777559,3328.0,9947778.0,11628648.0,2,0,times/month,ach_times_month
1969,1759639,4225273,17873795,1759639,4266614,2018-12-19,2018-12-19 00:00:00,38000280,8,NaN,NaN,NaN,44777559,1950.0,9947783.0,11628653.0,2,0,times/month,ach_times_month
1970,1760220,4225273,17870127,1760220,4266614,2018-08-28,2018-08-28 00:00:00,38000280,5,NaN,NaN,NaN,44777559,7776.0,9147443.0,10695938.0,2,0,times/month,ach_times_month


In [25]:
al_df = al_df.loc[:,['person_id','observation_date','observation_source_value']]
al_df.columns = ['person_id','observation_date','alcohol_habit']

In [26]:
treat = pd.merge(real_cg,al_df,how='left',on=['person_id'])

In [27]:
treat

,person_id,gender,year_of_birth,date,observation_date,alcohol_habit
0,69,1,1945,2000-01-03,NaN,NaN
1,115,0,1949,2000-01-18,NaN,NaN
2,150,1,1928,2006-04-17,NaN,NaN
3,157,0,1946,2001-01-15,NaN,NaN
4,270,0,1962,2000-01-12,NaN,NaN
...,...,...,...,...,...,...
13482,1760388,1,1971,2014-05-17,NaN,NaN
13483,1760747,1,1961,2007-03-02,NaN,NaN
13484,1760757,1,1977,1999-11-29,NaN,NaN
13485,1760837,1,1972,1999-12-15,2009-03-18,1


In [28]:
#Counter(treat['alcohol_habit'])

In [29]:
treat.loc[:,list(treat.loc[:,treat.isnull().any()].columns)].isnull().sum()/(len(treat))*100

observation_date    85.378513
alcohol_habit       85.378513
dtype: float64

## BMI

In [30]:
bmi = pd.read_csv('measurement0722(3036277,3025315).csv')

bmi_df = pd.DataFrame(bmi)

In [31]:
bmi_df

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value,care_site_source_value
0,68242,18,3025315,2013-01-17,2013-01-17 00:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
1,7795104,18,3036277,2013-01-17,2013-01-17 00:00:00,44818704,NaN,153.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS
2,7795105,18,3025315,2013-01-17,2013-01-17 00:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
3,9619505,18,3036277,2013-01-17,2013-01-17 12:00:00,44818704,NaN,153.0,0,NaN,NaN,NaN,1595.0,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS
4,9619506,18,3025315,2013-01-17,2013-01-17 12:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,1595.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607578,16771413,1760905,3025315,2006-12-10,2006-12-10 00:00:00,44818704,NaN,59.4,0,NaN,NaN,NaN,7154.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607579,16791060,1760905,3025315,2006-12-24,2006-12-24 00:00:00,44818704,NaN,59.0,0,NaN,NaN,NaN,1528.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607580,16973257,1760905,3025315,2006-12-31,2006-12-31 00:00:00,44818704,NaN,60.8,0,NaN,NaN,NaN,113.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607581,4686110,1760909,3036277,2006-08-01,2006-08-01 10:44:00,44818704,NaN,167.8,0,NaN,NaN,NaN,NaN,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS


In [32]:
bmi_df.sort_values(by=['person_id','measurement_date'])

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value,care_site_source_value
0,68242,18,3025315,2013-01-17,2013-01-17 00:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
1,7795104,18,3036277,2013-01-17,2013-01-17 00:00:00,44818704,NaN,153.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS
2,7795105,18,3025315,2013-01-17,2013-01-17 00:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,7667.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
3,9619505,18,3036277,2013-01-17,2013-01-17 12:00:00,44818704,NaN,153.0,0,NaN,NaN,NaN,1595.0,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS
4,9619506,18,3025315,2013-01-17,2013-01-17 12:00:00,44818704,NaN,60.0,0,NaN,NaN,NaN,1595.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,OS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1607576,4928798,1760905,3025315,2006-12-17,2006-12-17 00:00:00,44818704,NaN,61.4,0,NaN,NaN,NaN,900.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607579,16791060,1760905,3025315,2006-12-24,2006-12-24 00:00:00,44818704,NaN,59.0,0,NaN,NaN,NaN,1528.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607580,16973257,1760905,3025315,2006-12-31,2006-12-31 00:00:00,44818704,NaN,60.8,0,NaN,NaN,NaN,113.0,NaN,NaN,8KokmL3xmLiohzW5q6bvDrtteNvAJj5LA8MJnkM8r1gd,0,kg,NaN,NP
1607581,4686110,1760909,3036277,2006-08-01,2006-08-01 10:44:00,44818704,NaN,167.8,0,NaN,NaN,NaN,NaN,NaN,NaN,FZytTe58TunRd1jnykYHR3pTuQtM9D5mov5V3bea7zjx,0,cm,NaN,OS


In [33]:
bmi_df = bmi_df[['person_id','measurement_date','measurement_concept_id','value_as_number']]

In [34]:
height = bmi_df[bmi_df['measurement_concept_id']==3036277]
weight = bmi_df[bmi_df['measurement_concept_id']==3025315]

In [35]:
height

,person_id,measurement_date,measurement_concept_id,value_as_number
1,18,2013-01-17,3036277,153.0
3,18,2013-01-17,3036277,153.0
6,44,2016-05-24,3036277,150.0
8,49,2019-05-24,3036277,160.0
10,49,2019-05-24,3036277,160.0
...,...,...,...,...
1607564,1760694,2017-04-02,3036277,150.0
1607566,1760710,2011-07-15,3036277,169.8
1607568,1760792,2012-05-09,3036277,147.4
1607570,1760837,2018-07-23,3036277,158.9


In [36]:
del height['measurement_concept_id']

height.columns = ['person_id','observation_date','height']

del weight['measurement_concept_id']

weight.columns = ['person_id','observation_date','weight']

bmi_merge = pd.merge(height,weight,how = 'outer',on=['person_id','observation_date'])
bmi_merge['bmi'] = round(bmi_merge['weight']/((bmi_merge['height']/100)**2),2)
bmi_merge = bmi_merge.dropna(axis=0)

In [37]:
bmi_merge

,person_id,observation_date,height,weight,bmi
0,18,2013-01-17,153.0,60.0,25.63
1,18,2013-01-17,153.0,60.0,25.63
2,18,2013-01-17,153.0,60.0,25.63
3,18,2013-01-17,153.0,60.0,25.63
4,18,2013-01-17,153.0,60.0,25.63
...,...,...,...,...,...
2759575,1760694,2017-04-02,150.0,48.0,21.33
2759576,1760710,2011-07-15,169.8,69.4,24.07
2759577,1760792,2012-05-09,147.4,60.2,27.71
2759578,1760837,2018-07-23,158.9,62.4,24.71


In [38]:
del bmi_merge['height']
del bmi_merge['weight']

In [39]:
treat2 = pd.merge(treat, bmi_merge, how='left',on=['person_id','observation_date'])

In [40]:
treat2.loc[:,list(treat2.loc[:,treat2.isnull().any()].columns)].isnull().sum()/(len(treat2))*100

observation_date    39.000847
alcohol_habit       39.000847
bmi                 40.880610
dtype: float64

## Smoking Status

In [41]:
smoke = pd.read_csv('smoking0722(4222303,40766945).csv')

sm_df = pd.DataFrame(smoke)

sm_df.sort_values(by=['person_id','observation_date'],axis=0,inplace=True)

In [42]:
del sm_df['condition_concept_id']

In [43]:
Counter(sm_df['observation_concept_id'])

Counter({40766945: 3058, 4222303: 1150})

In [44]:
# 1 : 피우지 않는다 / 2 : 과거에 피웠으나 지금은 끊었다 / 3 : 현재도 피운다
sm_df = sm_df.astype({'observation_concept_id':str})

concept_id = {'4222303':1,'40766945':3}

sm_df['observation_concept_id']=sm_df['observation_concept_id'].replace(concept_id)

In [45]:
sm_df = sm_df[['person_id','observation_date','observation_concept_id']]

In [46]:
sm_df.columns = ['person_id','observation_date','smoking status']

In [47]:
sm_df

,person_id,observation_date,smoking status
0,1190,2016-08-28,3
1,1190,2016-08-28,3
2,1665,2015-12-29,1
3,2577,2018-10-08,1
3184,3179,2009-09-09,3
...,...,...,...
2103,1756845,2010-02-24,3
3181,1758943,2016-12-20,1
2104,1760220,2013-02-18,3
3182,1760220,2013-02-18,3


In [48]:
treat3 = pd.merge(treat2,sm_df,how='left',on=['person_id','observation_date'])

In [49]:
treat3.loc[:,list(treat3.loc[:,treat3.isnull().any()].columns)].isnull().sum()/(len(treat3))*100

observation_date    31.410256
alcohol_habit       31.410256
bmi                 33.589744
smoking status      60.373704
dtype: float64

## date(건강검진 날짜)가 없는 기록에 대해 삭제

In [50]:
treat3 = treat3.dropna(subset = ['observation_date'])

In [51]:
treat3

,person_id,gender,year_of_birth,date,observation_date,alcohol_habit,bmi,smoking status
30,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN
31,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN
32,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN
33,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN
34,1562,0,1951,2013-01-15,2015-08-31,1,27.15,NaN
...,...,...,...,...,...,...,...,...
36637,1759469,0,1946,2011-03-10,2017-10-01,2,26.58,NaN
36640,1759639,0,1945,2005-11-03,2015-08-20,2,21.53,NaN
36641,1759639,0,1945,2005-11-03,2018-12-19,2,19.68,NaN
36650,1760220,0,1953,2004-04-06,2018-08-28,2,NaN,3.0


## y_label

In [52]:
# 위염 -> 위암 발전환자
#to_cancer

In [53]:
cg_no = treat3[~treat3['person_id'].isin(to_cancer)] # 위염환자
cg_yes = treat3[treat3['person_id'].isin(to_cancer)] # 위암환자

In [54]:
print(len(cg_no))
print(len(cg_yes))

24965
180


In [55]:
cg_no['cancer'] = 0
cg_yes['cancer'] = 1

<ipython-input-55-f80891845757>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cg_no['cancer'] = 0
<ipython-input-55-f80891845757>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cg_yes['cancer'] = 1


In [56]:
cg_df = (pd.concat([cg_no, cg_yes])).sample(frac=1)

In [57]:
cg_df

,person_id,gender,year_of_birth,date,observation_date,alcohol_habit,bmi,smoking status,cancer
6006,149378,0,1948,2007-01-11,2018-09-04,1,23.44,3.0,0
1384,49313,0,1959,2014-05-19,2014-04-22,4,24.09,3.0,0
13541,423414,1,1964,2012-10-22,2012-02-29,2,16.02,NaN,0
20110,730213,0,1957,2015-01-22,2018-09-20,1,25.95,NaN,0
7915,191565,0,1971,2016-07-22,2019-09-09,1,24.27,3.0,0
...,...,...,...,...,...,...,...,...,...
36158,1750330,1,1961,2013-06-25,2015-10-27,2,25.39,3.0,0
561,20370,0,1950,2007-01-29,2014-12-27,2,26.18,NaN,0
26262,1192612,0,1945,2000-04-24,2019-03-17,2,24.31,NaN,0
16990,581660,0,1980,2011-07-06,2012-11-17,1,17.88,3.0,0


In [58]:
Counter(cg_df['cancer'])

Counter({0: 24965, 1: 180})

## EGD (위내시경)

In [59]:
egd = pd.read_csv('EGD0722(4294382).csv')

egd = pd.DataFrame(egd)

egd.sort_values(by=['person_id','procedure_date'],axis=0,inplace=True)

In [60]:
egd

,person_id,condition_concept_id,procedure_concept_id,procedure_date
0,150,4225273,4294382,2006-04-04
1,150,4225273,4294382,2006-04-04
4,157,4225273,4294382,2016-04-12
5,157,4225273,4294382,2016-04-12
3,157,4225273,4294382,2016-10-25
...,...,...,...,...
23806,1760379,4225273,4294382,2010-12-01
23809,1760388,4225273,4294382,2014-05-17
23810,1760388,4225273,4294382,2019-12-03
23811,1760747,4225273,4294382,2005-09-22


In [61]:
egd['date'] = pd.to_datetime(egd['procedure_date'])
egd['date'] = egd['date'].dt.year

In [63]:
egd_group = egd.groupby([egd['person_id'],egd['date']]).count()

In [64]:
egd_group = pd.DataFrame(egd_group)
egd_group = egd_group.reset_index()

In [67]:
egd_group = egd_group[['person_id','date','condition_concept_id']]

In [68]:
egd_group.columns = ['person_id','date1','EGD']

In [69]:
egd_group

,person_id,date1,EGD
0,150,2006,2
1,157,2016,5
2,374,2005,2
3,374,2006,1
4,495,2006,1
...,...,...,...
15514,1760379,2010,3
15515,1760388,2014,1
15516,1760388,2019,1
15517,1760747,2005,1


In [70]:
print(Counter(egd_group['EGD']))

Counter({1: 9684, 2: 4405, 3: 836, 4: 373, 5: 98, 6: 76, 7: 23, 8: 11, 9: 10, 10: 2, 14: 1})


In [71]:
cg_df['date1'] = pd.to_datetime(cg_df['observation_date']) 
cg_df['date1'] = cg_df['date1'].dt.year

# 건강검진 해당 연도에 내시경 검사를 받은 횟수 추가를 위해 건강검진 연도 Column 생성

In [72]:
# 위내시경을 안 받은 사람 포함
cg_df1 = pd.merge(cg_df,egd_group,how='left',on=['person_id','date1'])

In [73]:
cg_df1.loc[:,list(cg_df1.loc[:,cg_df1.isnull().any()].columns)].isnull().sum()/(len(cg_df1))*100

bmi                3.177570
smoking status    42.227083
EGD               45.679061
dtype: float64

In [74]:
cg_df1.sort_values(by=['person_id','observation_date'],axis=0,inplace=True)

In [75]:
cg_df1

,person_id,gender,year_of_birth,date,observation_date,alcohol_habit,bmi,smoking status,cancer,date1,EGD
191,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN
3637,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN
12781,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN
22323,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN
1186,1562,0,1951,2013-01-15,2015-08-31,1,27.15,NaN,0,2015,1.0
...,...,...,...,...,...,...,...,...,...,...,...
21658,1759469,0,1946,2011-03-10,2017-10-01,2,26.58,NaN,0,2017,1.0
11865,1759639,0,1945,2005-11-03,2015-08-20,2,21.53,NaN,0,2015,NaN
10421,1759639,0,1945,2005-11-03,2018-12-19,2,19.68,NaN,0,2018,NaN
838,1760220,0,1953,2004-04-06,2018-08-28,2,NaN,3.0,0,2018,NaN


## Age (건강검진 당시 나이)

### DataFrame 명 변경 확인.

In [78]:
new_cg = cg_df1

In [79]:
new_cg['age'] = new_cg['date1']-new_cg['year_of_birth']+1

In [80]:
new_cg

,person_id,gender,year_of_birth,date,observation_date,alcohol_habit,bmi,smoking status,cancer,date1,EGD,age
191,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN,63
3637,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN,63
12781,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN,63
22323,1562,0,1951,2013-01-15,2013-09-23,1,26.56,NaN,0,2013,NaN,63
1186,1562,0,1951,2013-01-15,2015-08-31,1,27.15,NaN,0,2015,1.0,65
...,...,...,...,...,...,...,...,...,...,...,...,...
21658,1759469,0,1946,2011-03-10,2017-10-01,2,26.58,NaN,0,2017,1.0,72
11865,1759639,0,1945,2005-11-03,2015-08-20,2,21.53,NaN,0,2015,NaN,71
10421,1759639,0,1945,2005-11-03,2018-12-19,2,19.68,NaN,0,2018,NaN,74
838,1760220,0,1953,2004-04-06,2018-08-28,2,NaN,3.0,0,2018,NaN,66


In [81]:
del new_cg['date1']
del new_cg['year_of_birth']

In [83]:
new_cg = new_cg[['person_id','gender','age','EGD','bmi','alcohol_habit','smoking status','cancer','date', 'observation_date']]

In [84]:
new_cg.loc[:,list(new_cg.loc[:,new_cg.isnull().any()].columns)].isnull().sum()/(len(new_cg))*100

EGD               45.679061
bmi                3.177570
smoking status    42.227083
dtype: float64

In [85]:
new_cg.to_csv('pre_kd.csv',mode='w',index=False) # 시계열 데이터